In [ ]:
#import dependencies
#maybe comment this out a little more
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import os
from scipy.stats import linregress
from scipy import stats
import scipy.stats as stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
#Bernt's code starts here

In [ ]:
# Importing necessary dataframes

birthrate_csv = "../project-one/Resources/children_per_woman_total_fertility.csv"
birthrate_df = pd.read_csv(birthrate_csv)
birthrate_df.head()

In [ ]:
co2_csv = "../project-one/Resources/co2_emissions_tonnes_per_person.csv"
co2_df = pd.read_csv(co2_csv)
co2_df.head()

In [ ]:
income_csv = "../project-one/Resources/income_per_person_gdppercapita_ppp_inflation_adjusted.csv"
income_df = pd.read_csv(income_csv)
income_df.head()

In [ ]:
population_csv = "../project-one/Resources/population_total.csv"
population_df = pd.read_csv(population_csv)
population_df.head()

In [1]:
#Bernt's code ends here

In [2]:
#Molly's code starts here

In [3]:
#Molly's code ends here

In [4]:
#Kaan's code starts here

In [5]:
#Kaan's code ends here

In [6]:
#Vallie's code starts here

In [ ]:
#Vallie's code ends here